In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.base import BaseEstimator, TransformerMixin
from pathlib import Path



In [ ]:
BASE_DIR = Path(r"C:\Felix\Projektarbeit_f\Daten")  # <-- Hier anpassen

In [ ]:
# ========== 1. Rohdaten laden ==========

df_lines = pd.read_parquet(BASE_DIR / "transaction_lines_train_2.parquet")
df_trans = pd.read_parquet(BASE_DIR / "transactions_train_2.parquet")
df_products = pd.read_csv(BASE_DIR / "products.csv")
df_stores = pd.read_csv(BASE_DIR / "stores.csv")

df_lines_test = pd.read_parquet(BASE_DIR / "transaction_lines_test_2.parquet")
df_trans_test = pd.read_parquet(BASE_DIR / "transactions_test_2.parquet")

In [ ]:

# ========== 2. Vorbereitung: Produkte + Stores ==========
df_lines_merged = df_lines.merge(df_products, left_on="product_id", right_on="id", suffixes=('', '_product'))
df_lines_merged.drop(columns=["id_product", "valid_from", "valid_to"], inplace=True)

df_features = df_lines_merged.groupby("transaction_id").agg({
    "sales_price": ["sum", "mean", "max", "min"],
    "camera_certainty": ["mean", "min"],
    "was_voided": ['sum', 'mean'],
    "camera_product_similar": ['sum', 'mean'],
    "price": ['sum', 'mean'],
    "category": ['nunique'],
    "weight": ["sum", "mean"],
    "sold_by_weight": ["sum"],
    "age_restricted": ['mean'],
    "product_id": ["count"],
    "base_product_id": ["nunique"]
})

# Nur FRAUD und NORMAL Daten verwenden
df_trans = df_trans[df_trans['label'].isin(['NORMAL', 'FRAUD'])].copy()

df_features.columns = ["_".join(col).strip() for col in df_features.columns.values]
df_features.reset_index(inplace=True)

df_model = df_trans.merge(df_features, left_on="id", right_on="transaction_id")
df_model = df_model.merge(df_stores, left_on="store_id", right_on="id", suffixes=('', '_store'))

df_model.drop(columns=["transaction_id", "id_store"], inplace=True)


In [ ]:
# Drop Spalte customer_feedback mit zu vielen NaNs
if "customer_feedback" in df_model.columns:
    df_model.drop(columns=["customer_feedback"], inplace=True)

In [ ]:
# ========== 3. Transformer definieren ==========
class FeatureEngineeringTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, lines_with_category=None):
        self.lines_with_category = lines_with_category

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        df = X.copy()
        df['transaction_start'] = pd.to_datetime(df['transaction_start'])
        df['transaction_end'] = pd.to_datetime(df['transaction_end'])
        df['has_cash_payment'] = (df['payment_medium'] == 'CASH').astype(int)
        df['average_price_per_article'] = df['total_amount'] / (df['n_lines'] + 1e-5)
        df['transaction_duration_seconds'] = (df['transaction_end'] - df['transaction_start']).dt.total_seconds()
        df['articles_per_minute'] = df['n_lines'] / (df['transaction_duration_seconds'] / 60 + 1e-5)
        df['voided_articles_ratio'] = df['was_voided_sum'] / (df['n_lines'] + 1e-5)

        # Datumsangaben transformieren
        df['month'] = df['transaction_start'].dt.month
        df['hour'] = df['transaction_start'].dt.hour
        df['weekday'] = df['transaction_start'].dt.weekday

        df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
        df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

        df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
        df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

        df['weekday_sin'] = np.sin(2 * np.pi * df['weekday'] / 7)
        df['weekday_cos'] = np.cos(2 * np.pi * df['weekday'] / 7)

        # one hot encoding
        pd.get_dummies(df, columns=['urbanization'], prefix='urb')

        # drop categoric and date columns
        df.drop(columns=['state', "opening_date", "sco_introduction", 'transaction_start', 'transaction_end',
                         "payment_medium", "month", "hour", "cash_desk"], inplace=True)

        if self.lines_with_category is not None:
            df_lines = self.lines_with_category.copy()
            snack_lines = df_lines[df_lines['category'] == 'SNACKS']
            snack_counts = snack_lines.groupby('transaction_id').size().reset_index(name='snack_count')
            df = df.merge(snack_counts, left_on='id', right_on='transaction_id', how='left')
            df['snack_count'] = df['snack_count'].fillna(0).astype(int)
            df['snack_share'] = df['snack_count'] / (df['n_lines'] + 1e-5)
            df.drop(columns=['transaction_id'], inplace=True, errors='ignore')

        return df

In [ ]:
# ========== 4. Trainingstransformer anwenden ==========
lines_with_category_train = df_lines.merge(
    df_products[['id', 'category', 'sold_by_weight', 'age_restricted']],
    left_on='product_id',
    right_on='id',
    suffixes=('_line', '_product')
)

transformer = FeatureEngineeringTransformer(lines_with_category=lines_with_category_train)


In [ ]:
# Feature Engineering auf Trainingsdaten
df_ready_train = transformer.transform(df_model)

# Label-Spalte erstellen
df_ready_train['label_fraud_bin'] = (df_ready_train['label'] == 'FRAUD').astype(int)

# Label-Spalte, Payment Medium Spalte kategorisch entfernen
df_ready_train.drop(columns=["label"], inplace=True)

'''
# === NEU: Filterung von echten Fraud-F√§llen mit relevantem Schaden ===
if "damage" in df_ready_train.columns:
    is_fraud = df_ready_train["label_fraud_bin"] == 1
    is_fraud_with_damage = df_ready_train["damage"] >= 0.05 * df_ready_train["total_amount"]

    # Statistik ausgeben
    print("üìä Anzahl aller FRAUD-Zeilen:", is_fraud.sum())
    print("‚úÖ Davon mit ‚â•5% Schaden:", (is_fraud & is_fraud_with_damage).sum())

    # Filter anwenden: normale behalten + fraud mit gen√ºgend Schaden
    is_normal = df_ready_train["label_fraud_bin"] == 0
    df_ready_train = df_ready_train[is_normal | (is_fraud & is_fraud_with_damage)].copy()

else:
    print("‚ö†Ô∏è Spalte 'damage' nicht vorhanden ‚Äì kein Fraud-Filter angewendet.")
'''

üìä Anzahl aller FRAUD-Zeilen: 4766
‚úÖ Davon mit ‚â•5% Schaden: 3247


In [ ]:
# Export f√ºr Modelltraining
df_ready_train.to_csv(BASE_DIR / "df_model_ready_train.csv", index=False)

In [ ]:
# ========== 5. Testdaten analog verarbeiten ==========
# Join + Feature Engineering
df_lines_test_merged = df_lines_test.merge(df_products, left_on="product_id", right_on="id", suffixes=('', '_product'))
df_lines_test_merged.drop(columns=["id_product", "valid_from", "valid_to"], inplace=True)

df_features_test = df_lines_test_merged.groupby("transaction_id").agg({
     "sales_price": ["sum", "mean", "max", "min"],
    "camera_certainty": ["mean", "min"],
    "was_voided": ['sum', 'mean'],
    "camera_product_similar": ['sum', 'mean'],
    "price": ['sum', 'mean'],
    "category": ['nunique'],
    "weight": ["sum", "mean"],
    "sold_by_weight": ["sum"],
    "age_restricted": ['mean'],
    "product_id": ["count"],
    "base_product_id": ["nunique"]
})

df_features_test.columns = ["_".join(col).strip() for col in df_features_test.columns.values]
df_features_test.reset_index(inplace=True)

df_model_test = df_trans_test.merge(df_features_test, left_on="id", right_on="transaction_id")
df_model_test = df_model_test.merge(df_stores, left_on="store_id", right_on="id", suffixes=('', '_store'))
df_model_test.drop(columns=["transaction_id", "id_store"], inplace=True)

# Optional Spalten entfernen
if "customer_feedback" in df_model_test.columns:
    df_model_test.drop(columns=["customer_feedback"], inplace=True)

# Feature Engineering Testdaten
lines_with_category_test = df_lines_test.merge(
    df_products[['id', 'category', 'sold_by_weight', 'age_restricted']],
    left_on='product_id',
    right_on='id',
    suffixes=('_line', '_product')
)

transformer_test = FeatureEngineeringTransformer(lines_with_category=lines_with_category_test)
df_ready_test = transformer_test.transform(df_model_test)

# Export Testdaten f√ºr Modell
df_ready_test.to_csv(BASE_DIR / "df_model_ready_test.csv", index=False)

# In zwei H√§lften splitten
half = len(df_ready_test) // 2
df_test_1 = df_ready_test.iloc[:half].copy()
df_test_2 = df_ready_test.iloc[half:].copy()

# Beide Teile speichern ‚Äì Pfad mit BASE_DIR
df_test_1.to_csv(BASE_DIR / "df_model_ready_test_part1.csv", index=False)
df_test_2.to_csv(BASE_DIR / "df_model_ready_test_part2.csv", index=False)

print(f"‚úÖ Testdaten wurden halbiert: {len(df_test_1)} + {len(df_test_2)} Zeilen gespeichert.")


‚úÖ Testdaten wurden halbiert: 410064 + 410065 Zeilen gespeichert.
